In [32]:
import glob
import multiprocessing
import subprocess
import os, shutil

In [51]:
docker_id = '1b00558574a6'
n_proc = 32

#-r (positional arg)
pre_dir = '/mnt/ibm_lg/daniel_le/data/longreads/c3poa_testdata/output/pre'
r_vals = glob.glob(f'{pre_dir}/*/*/R2C2_raw_reads.fastq')
# -p, -o (create custom output)
main_dir = '/mnt/ibm_lg/daniel_le/data/longreads/c3poa_testdata/output/main'
main_subdirs = [f'{main_dir}/' + '_'.join(x.split('/')[-3:-1]) for x in r_vals]
for x in main_subdirs:
    if os.path.exists(x):
        shutil.rmtree(x)
    os.mkdir(x)
    os.mkdir(f'{x}/tmp')
p_vals = [f'{x}/tmp' for x in main_subdirs]
o_vals = [f'{x}/css.fasta' for x in main_subdirs]
# -m, -l, -d, -c (positional args)
m_val = '/mnt/ibm_lg/daniel_le/data/longreads/c3poa_testdata/input/NUC.4.4.mat'
# l_val = 1000
# d_val = 500
c_val = '/mnt/ibm_lg/daniel_le/data/longreads/c3poa_testdata/input/config'


In [ ]:
%%time
# docker command
def docker_call(r_val, p_val, o_val):
    log_fn = f'{p_val}/log.txt'
    try: 
        cmnd = ' '.join(['docker',
                         'exec',
                         docker_id,
                         'python3',
                         'C3POa.py',
                         '-r',
                         r_val,
                         '-p',
                         p_val,
                         '-o',
                         o_val,
                         '-m',
                         m_val,
#                          '-l',
#                          str(l_val),
#                          '-d',
#                          str(d_val),
                         '-c',
                         c_val,
                        ])
        messages = [cmnd]
        output = subprocess.Popen(
            cmnd, 
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            shell=True,
            universal_newlines=True)
        out,err = output.communicate()

    except:
        print('FAILED:', cmnd)
        messages = [cmnd,'\n',out,'\n',err]
        
    with open(log_fn, 'w') as outfile:
        for message in messages:
            for line in message:
                outfile.write(line)

p = multiprocessing.Pool(processes=n_proc)
try:
    # testing
#     p.starmap(docker_call, zip(['/mnt/ibm_lg/daniel_le/data/longreads/c3poa_testdata/output/pre/7/UMI_Splint_3/R2C2_raw_reads.fastq'],
#                                ['/mnt/ibm_lg/daniel_le/data/longreads/c3poa_testdata/output/main/7_UMI_Splint_3/tmp'],
#                                ['/mnt/ibm_lg/daniel_le/data/longreads/c3poa_testdata/output/main/7_UMI_Splint_3/css.fasta']
#                           ))
    p.starmap(docker_call, zip(r_vals,
                       p_vals,
                       o_vals,
                      ))
finally:
    p.close()
    p.join()
    


In [54]:
%%time
# cat outputs
with open(f'{main_dir}/merged_ccs.fasta', 'w') as outfile:
    for fname in o_vals:
        with open(fname) as infile:
            for line in infile:
                outfile.write(line)

CPU times: user 143 ms, sys: 116 ms, total: 259 ms
Wall time: 412 ms


In [38]:
[print(x,'\n',y,'\n',z) for x,y,z in zip(r_vals,
                           p_vals,
                           o_vals,
                          )]

/mnt/ibm_lg/daniel_le/data/longreads/c3poa_testdata/output/pre/7/UMI_Splint_3/R2C2_raw_reads.fastq 
 /mnt/ibm_lg/daniel_le/data/longreads/c3poa_testdata/output/main/7_UMI_Splint_3/tmp 
 /mnt/ibm_lg/daniel_le/data/longreads/c3poa_testdata/output/main/7_UMI_Splint_3/css.fasta
/mnt/ibm_lg/daniel_le/data/longreads/c3poa_testdata/output/pre/7/UMI_Splint_2/R2C2_raw_reads.fastq 
 /mnt/ibm_lg/daniel_le/data/longreads/c3poa_testdata/output/main/7_UMI_Splint_2/tmp 
 /mnt/ibm_lg/daniel_le/data/longreads/c3poa_testdata/output/main/7_UMI_Splint_2/css.fasta
/mnt/ibm_lg/daniel_le/data/longreads/c3poa_testdata/output/pre/7/UMI_Splint_1/R2C2_raw_reads.fastq 
 /mnt/ibm_lg/daniel_le/data/longreads/c3poa_testdata/output/main/7_UMI_Splint_1/tmp 
 /mnt/ibm_lg/daniel_le/data/longreads/c3poa_testdata/output/main/7_UMI_Splint_1/css.fasta
/mnt/ibm_lg/daniel_le/data/longreads/c3poa_testdata/output/pre/6/UMI_Splint_3/R2C2_raw_reads.fastq 
 /mnt/ibm_lg/daniel_le/data/longreads/c3poa_testdata/output/main/6_UMI_Spli

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]